# 선형 회귀 학습 실습

이 노트북에서는 선형 회귀의 최소제곱법과 경사하강법을 단계별로 실습합니다.

## 사용할 수식

**모델:**

$$\hat{y} = \theta_0 + \theta_1 x = \boldsymbol{\theta}^T \mathbf{a}$$

여기서 $\mathbf{a} = (1, x)^T$는 augmented 입력, $\boldsymbol{\theta} = (\theta_0, \theta_1)^T$는 파라미터입니다.

**손실 함수:** Sum of Squared Errors

$$J(\boldsymbol{\theta}) = \sum_{k=1}^{N} (y_k - \hat{y}_k)^2 = \|\mathbf{y} - \mathbf{A}\boldsymbol{\theta}\|^2$$

**정규방정식 (Normal Equation):**

$$\boldsymbol{\theta}^* = (\mathbf{A}^T \mathbf{A})^{-1} \mathbf{A}^T \mathbf{y}$$

**경사하강법 (Gradient Descent):**

$$\nabla J = 2 \, \mathbf{A}^T (\mathbf{A}\boldsymbol{\theta} - \mathbf{y})$$

$$\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} - \alpha \, \nabla J$$


---
## Step 0: 라이브러리 임포트


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 그래프 설정
plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['axes.unicode_minus'] = False


---
## Step 1: 데이터 생성

$$y = 0.1 + 0.3 \, x + \mathcal{N}(0,\, 0.1)$$

으로부터 $N = 100$개 샘플을 생성합니다.


In [ ]:
# 참값 (ground truth)
N      = 100
theta0 = 0.1     # 절편
theta1 = 0.3     # 기울기

# 데이터 생성
x = np.random.normal(0.0, 1, N).reshape(-1, 1)
y = theta0 + theta1 * x
y = y + np.random.normal(0.0, 0.1, N).reshape(-1, 1)

print(f"x shape: {x.shape}")
print(f"y shape: {y.shape}")
print(f"참값: θ₀ = {theta0}, θ₁ = {theta1}")

# 데이터 시각화
plt.figure()
plt.scatter(x, y, c='#e53935', s=30, alpha=0.7, edgecolors='white', linewidths=0.5)
plt.xlabel('x', fontsize=13)
plt.ylabel('y', fontsize=13)
plt.title('Generated Data: $y = 0.1 + 0.3x + \\epsilon$', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


---
## Step 2: 정규방정식으로 풀기 (Least Squares)

Augmented 행렬 $\mathbf{A}$를 구성하고, 정규방정식을 적용합니다.

$$\mathbf{A} = \begin{pmatrix} 1 & x_1 \\ 1 & x_2 \\ \vdots & \vdots \\ 1 & x_N \end{pmatrix}, \qquad \boldsymbol{\theta}^* = (\mathbf{A}^T \mathbf{A})^{-1} \mathbf{A}^T \mathbf{y}$$


In [ ]:
# Augmented 행렬 A 구성: [1, x]
A = np.hstack([x**0, x])
A = np.asmatrix(A)

# 정규방정식: θ = (AᵀA)⁻¹ Aᵀy
theta_ls = np.array((A.T * A).I * A.T * y)

print(f"A shape: {A.shape}")
print(f"\n정규방정식 결과:")
print("=" * 40)
print(f"  θ₀ (절편)  = {theta_ls[0, 0]:.6f}  (참값: {theta0})")
print(f"  θ₁ (기울기) = {theta_ls[1, 0]:.6f}  (참값: {theta1})")
print("=" * 40)


In [ ]:
# 결과 시각화
plt.figure()
plt.scatter(x, y, c='#e53935', s=30, alpha=0.7, edgecolors='white', linewidths=0.5, label='data')
plt.plot(x, theta_ls[1, 0] * x + theta_ls[0, 0], 'b-', linewidth=2.5,
         label=f'LS: $y = {theta_ls[0,0]:.3f} + {theta_ls[1,0]:.3f}x$')
plt.xlabel('x', fontsize=13)
plt.ylabel('y', fontsize=13)
plt.title('Least Squares Solution', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


---
## Step 3: 경사하강법으로 풀기 (Gradient Descent)

손실 함수의 그래디언트를 이용하여 $\boldsymbol{\theta}$를 반복적으로 갱신합니다.

$$\nabla J = 2 \, \mathbf{A}^T (\mathbf{A}\boldsymbol{\theta} - \mathbf{y})$$

$$\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} - \alpha \, \nabla J$$


In [ ]:
# 초기 파라미터 (랜덤)
theta = np.random.randn(2, 1)
theta = np.asmatrix(theta)

# 하이퍼파라미터
alpha  = 0.00001     # 학습률
n_iter = 3000        # 반복 횟수
theta_history = []

print(f"초기 θ = ({theta[0,0]:.4f}, {theta[1,0]:.4f})")
print(f"학습률 α = {alpha}")
print(f"반복 횟수 = {n_iter}")
print()

# 경사하강법 반복
for i in range(n_iter):
    df    = 2 * (A.T * A * theta - A.T * y)    # ∇J = 2Aᵀ(Aθ - y)
    theta = theta - alpha * df                   # θ ← θ - α∇J
    theta_history.append(np.array(theta)[:, 0])

    if i % 500 == 0 or i == n_iter - 1:
        loss = float(np.sum(np.array(y - A * theta) ** 2))
        print(f"iter={i:5d}  θ₀={theta[0,0]:+.6f}  θ₁={theta[1,0]:+.6f}  loss={loss:.6f}")

print()
print("최종 결과:")
print("=" * 40)
print(f"  θ₀ = {theta[0,0]:.6f}  (참값: {theta0})")
print(f"  θ₁ = {theta[1,0]:.6f}  (참값: {theta1})")
print("=" * 40)


In [ ]:
# 회귀 직선 시각화
plt.figure()
plt.scatter(x, y, c='#e53935', s=30, alpha=0.7, edgecolors='white', linewidths=0.5, label='data')
plt.plot(x, float(theta[1, 0]) * x + float(theta[0, 0]), 'b-', linewidth=2.5,
         label=f'GD: $y = {float(theta[0,0]):.3f} + {float(theta[1,0]):.3f}x$')
plt.xlabel('x', fontsize=13)
plt.ylabel('y', fontsize=13)
plt.title('Gradient Descent Solution', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# θ 수렴 과정 시각화
theta_history = np.array(theta_history)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# θ₀ 수렴
axes[0].plot(np.arange(n_iter), np.ones(n_iter) * theta0,
             '#e53935', linewidth=2, linestyle='--', label=f'참값 = {theta0}')
axes[0].plot(theta_history[:, 0], '#1a73e8', linewidth=1.5, label='$\\theta_0$ (GD)')
axes[0].set_xlabel('Iteration', fontsize=12)
axes[0].set_ylabel('$\\theta_0$', fontsize=13)
axes[0].set_title('$\\theta_0$ 수렴 과정', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# θ₁ 수렴
axes[1].plot(np.arange(n_iter), np.ones(n_iter) * theta1,
             '#e53935', linewidth=2, linestyle='--', label=f'참값 = {theta1}')
axes[1].plot(theta_history[:, 1], '#1a73e8', linewidth=1.5, label='$\\theta_1$ (GD)')
axes[1].set_xlabel('Iteration', fontsize=12)
axes[1].set_ylabel('$\\theta_1$', fontsize=13)
axes[1].set_title('$\\theta_1$ 수렴 과정', fontsize=13, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


---
## 전체 과정 요약

| 단계 | 수식 | 설명 |
|:---|:---|:---|
| 데이터 | $y = 0.1 + 0.3x + \epsilon$ | $N=100$, $\epsilon \sim \mathcal{N}(0, 0.1)$ |
| Augmented 행렬 | $\mathbf{A} = [\mathbf{1}, \mathbf{x}]$ | shape: $(N, 2)$ |
| 정규방정식 | $\boldsymbol{\theta}^* = (\mathbf{A}^T\mathbf{A})^{-1}\mathbf{A}^T\mathbf{y}$ | 해석적 해 (closed-form) |
| 경사하강법 | $\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} - \alpha \cdot 2\mathbf{A}^T(\mathbf{A}\boldsymbol{\theta} - \mathbf{y})$ | 반복적 최적화 |


---
## 연습 문제

아래 셀에서 학습률(`alpha`)이나 반복 횟수(`n_iter`)를 변경해보고, 수렴 과정이 어떻게 달라지는지 확인해보세요.

1. `alpha = 0.0001`로 키우면 어떻게 되나요?
2. `n_iter = 100`으로 줄이면 수렴하나요?
3. 참값을 `theta0 = 2.0`, `theta1 = -0.5`로 바꾸고 처음부터 다시 해보세요.


In [ ]:
# 여기에 코드를 작성하세요


